<a href="https://colab.research.google.com/github/polis-community/red-dwarf/blob/main/docs/notebooks/strip-pass-votes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet --no-cache-dir git+https://github.com/polis-community/red-dwarf.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 168.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 145.0 MB/s eta 0:00:00


In [2]:
CONVOS = {
    # Topic: What were the most significant developments in tech and politics in 2018?
    # 5 groups, 65 ptpts (56 grouped), 43 comments (open)
    "tech-politics-2018": {
        "report_id": "r2dfw8eambusb8buvecjt",
        "convo_id": "6jrufhr6dp",
    },
    # Topic: How should we operate vehicle-for-hire, e.g. Uber, Lyft and taxis in Toronto?
    # 2 groups, 47 ptpts (36 grouped), 69 comments (open)
    "rideshare-toronto": {
        "report_id": "r8xhmkwp6shm9yfermteh",
        "convo_id": "7vampckwrh",
    },
}

In [3]:
# Set up interactive widgets.

from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from random import random

checkbox = widgets.Checkbox(
    value=False,
    description='Remove pass votes',
    indent=True,
)
random_val = widgets.FloatText()
button = widgets.Button(description="Re-run Clustering")

def on_button_click(_):
    random_val.value = random()
button.on_click(on_button_click)

In [6]:
from reddwarf.polis import PolisClient
from reddwarf.data_presenter import DataPresenter

report_id = CONVOS["tech-politics-2018"]["report_id"]
print(f"Loading data from report: https://pol.is/report/{report_id}")
print()
print("Run with or without pass votes, to see the algorithmic effect of including pass votes.")

# Display the checkboxes.
display(widgets.HBox([checkbox, button]))

# To see the consequences of not having pass/neutral/zero votes
def cluster_and_plot(do_strip_pass_votes, dummy_var):
    client = PolisClient()
    client.load_data(polis_id=report_id)
    if do_strip_pass_votes:
        client.votes = []
        client.load_votes_data(data=[v for v in client.data_loader.votes_data if v["vote"] != 0])
    client.get_matrix(is_filtered=True)
    client.run_pca()
    client.find_optimal_k()

    presenter = DataPresenter(client=client)
    presenter.render_optimal_cluster_figure()

# Render the interactive widget
widgets.interactive_output(cluster_and_plot, {'do_strip_pass_votes': checkbox, 'dummy_var': random_val})

Loading data from report: https://pol.is/report/r2dfw8eambusb8buvecjt

Run with or without pass votes, to see the algorithmic effect of including pass votes.


Output()